In [ ]:
# #Convex hull to stl example
# import numpy as np
# from scipy.spatial import ConvexHull
# from stl import mesh
# import pyvista as pv

# out="D:\\AMR_SIM\\AMR_POINT_CLOUD_WORKFLOW\\PYOUT\\"

# # Generate some random 3D points
# points = np.random.rand(60, 3)  # 30 random 3D points

# # Compute the convex hull
# hull = ConvexHull(points)

# vertices = points[hull.vertices]  # Vertices of the convex hull
# faces = hull.simplices  # Faces of the convex hull

# hull_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

# for i, face in enumerate(faces):
#     for j in range(3):
#         hull_mesh.vectors[i][j] = points[face[j]]  #


# hull_mesh.save(out+"convex_hull_mesh.stl")
# hull_mesh_STL=pv.read(out+"convex_hull_mesh.stl")


# pv.set_plot_theme("document")
# p = pv.Plotter()
# p.set_background("white")
# p.add_mesh(hull_mesh_STL, color="red")
# #p.add_mesh(plastic_STL, color="grey", opacity=0.1);
# p.window_size = 150, 150
# p.show(jupyter_backend='static');
# #p.view_yz()
# #p.screenshot(output_base+"images\\"+IMAGE_ID+'__3D_model.png') ;
# #p.screenshot(out+'root_binary_3D_model.png') ;

In [ ]:
import numpy as np
import pandas as pd
import tifffile
from scipy.ndimage import label
from skimage.measure import regionprops
import matplotlib.pyplot as plt
from skimage import data, filters, measure, morphology
from tqdm import tqdm 
from scipy.spatial import ConvexHull, Delaunay
import sys
import seaborn as sns
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from pyntcloud import PyntCloud
from stl import mesh
import pyvista as pv
# from vedo import *

In [ ]:
#!pip install numpy==1.26.4 #numpy version matters when making the movies ??

In [ ]:
#!pip install pyntcloud

In [ ]:
path= "D:\\AMR_SIM\\AMR_POINT_CLOUD_WORKFLOW\\"
file_name="20241031 - Repeat - Pos5 - DLS - Video.czi 568.txtfiltered.csv"
img_name="20241031 - Repeat - Pos5 - DLS - Video.czi 568_4xmag_4nm_256_predictions2.tif"
out="D:\\AMR_SIM\\AMR_POINT_CLOUD_WORKFLOW\\PYOUT\\"

In [ ]:
prob_map = tifffile.imread(path+img_name)

# Step 2: Threshold the probability map to get a binary mask (example threshold 0.5)
threshold = 0.5
binary_mask = prob_map[0] > threshold

# Step 3: Label the connected components (ROIs)
labeled_mask, num_features = label(binary_mask)
tifffile.imsave(out+"labeled_mask.tif", labeled_mask)

print(f"Number of ROIs: {num_features}")

# Step 4: Extract ROIs and their properties
regions = regionprops(labeled_mask)

roi_indices = {}

# Example: Print bounding box of each ROI
for idx in tqdm(range(0,len(regions))):
    minz, miny, minx, maxz, maxy, maxx = regions[idx].bbox
    # print(f"ROI {regions[idx].label} - Bounding Box: ({minr}, {minc}, {minz}) to ({maxr}, {maxc}, {maxz})")
    
    if maxy - miny < 2:
        continue
    elif maxx - minx < 2:
        continue
    indices = np.argwhere(labeled_mask[minz:maxz,miny:maxy,minx:maxx] == regions[idx].label)  # Get the indices of each label
    indices[:,0]=indices[:,0]+minz
    indices[:,1]=indices[:,1]+miny
    indices[:,2]=indices[:,2]+minx
    roi_indices[regions[idx].label] = indices  # Store indices in a dictionary

In [ ]:
dx, dy, dz = 25, 25, 4  # Adjust based on actual voxel size

# Step 4: Convert indices to continuous coordinates
roi_continuous_coords = {}
for label, indices in roi_indices.items():
    # Convert indices to continuous coordinates
    continuous_coords = indices * np.array([dz, dy, dx])
    roi_continuous_coords[label] = continuous_coords

In [ ]:
hulls = {}
for roi in tqdm(roi_indices):
    if len(roi_continuous_coords[roi]) < 9:
        continue
    
    hull = ConvexHull(roi_continuous_coords[roi])
    if hull.volume < 1000000:
        continue
    hulls[roi] = hull

In [ ]:
def point_in_hull(point, hull, tolerance=1e-12):
    return all(
        (np.dot(eq[:-1], point) + eq[-1] <= tolerance)
        for eq in hull.equations)

In [ ]:
df = pd.read_csv(path+file_name)
df["z"] = df["z [nm]"]
df["x"] = df["x [nm]"]
df["y"] = df["y [nm]"]

df["z"] = df["z"] + 424 #why ????

In [ ]:
#VERY SLOW

# # Iterate through DataFrame rows and assign ROI number
# roi_assignment = []
# for _, row in tqdm(df.iterrows()):
#     point = np.array([row['z'], row['y'], row['x']])  # Convert to (z, y, x)
#     assigned_roi = None
    
#     # Check the point against each ROI's contour
#     for roi_number, hull in hulls.items():
#         if point_in_hull(point, hull):
#             assigned_roi = roi_number
#             break  # Stop checking once we find a match
    
#     roi_assignment.append(assigned_roi)

# # Add the assigned ROI numbers to the DataFrame
# df['roi'] = roi_assignment

In [ ]:
#df.to_csv(path+"JANS_OUTPUT.csv", index=False)
df = pd.read_csv(path+"JANS_OUTPUT.csv")


In [ ]:
df 

In [ ]:
roi_num = 609

# Step 2: Create the ConvexHull object
hull = hulls[roi_num]

points = hull.points

points = points[:, [2, 1, 0]]

datapoints = df[df["roi"] == roi_num]

# Plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Step 5: Plot the convex hull
for simplex in hull.simplices:
    # Each simplex is a triangle (facet of the convex hull)
    vertices = points[simplex]
    hull_face = Poly3DCollection([vertices], color='cyan', alpha=0.3)  # Semi-transparent color
    ax.add_collection3d(hull_face)
    
# Plot datapoints
ax.scatter(datapoints['x'], datapoints['y'], datapoints['z'], c='b', marker='o')

# Set limits to zoom in
# ax.set_xlim([datapoints.min().iloc[0]-1000, datapoints.max().iloc[0]+1000])
# ax.set_ylim([datapoints.min().iloc[1]-1000, datapoints.max().iloc[1]+1000])
# ax.set_zlim([datapoints.min().iloc[2]-100, datapoints.max().iloc[2]+100])

# Set labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Add legend
ax.legend()

plt.show()


In [ ]:
#CREATE UNIQUIE ROIS
UNIQUE_ROIS =np.unique(df["roi"])

## LOOP TO GET RENDER OF PROTEIN IN varicosities_3D_model 

In [ ]:
#deal with exporting data 
# https://stackoverflow.com/questions/49637221/how-to-export-a-point-cloud
#(re)create point cloud
#Loop through ROIs and create render
for i in UNIQUE_ROIS:
    print (i)
    roi_num = i
    hull = hulls[roi_num]

    points = hull.points

    points = points[:, [2, 1, 0]]

    datapoints = df[df["roi"] == roi_num]
    
    d = {'x': datapoints['x'],'y': datapoints['y'],'z': datapoints['z']}
    cloud = PyntCloud(pd.DataFrame(data=d))
    cloud.to_file(out+str(roi_num)+"output.ply")

    #hull to mesh
    vertices = points[hull.vertices]  # Vertices of the convex hull
    faces = hull.simplices  # Faces of the convex hull

    hull_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

    for i, face in enumerate(faces):
        for j in range(3):
            hull_mesh.vectors[i][j] = points[face[j]]  #


    hull_mesh.save(out+str(roi_num)+"convex_hull_mesh.stl")

    hull_mesh_STL=pv.read(out+str(roi_num)+"convex_hull_mesh.stl")
    points_ply=pv.read(out+str(roi_num)+"output.ply")



    pv.set_plot_theme("document")
    p = pv.Plotter()
    p.set_background("white")
    p.add_mesh(hull_mesh_STL, color="grey", opacity=0.1)
    p.add_points(points_ply, style='points_gaussian',point_size=2, color="red")
    #p.add_mesh(plastic_STL, color="grey", opacity=0.1);
    p.window_size = 500, 500
    p.show(jupyter_backend='static');
    p.screenshot(out+str(roi_num)+'varicosities_3D_model.png') ;

## LOOP TO GET CLUSTER OF PROTEIN IN varicosities_3D_model 

In [ ]:
#https://stackoverflow.com/questions/56062673/clustering-the-3d-points-when-given-the-x-y-z-coordinates-using-dbscan-algorithm
#see also 
#https://stackoverflow.com/questions/65902125/dbscan-on-3d-coordinates-doesnt-find-clusters
#fro scaling 
# import numpy as np
# import matplotlib.pyplot as plt
# #%matplotlib inline
# from mpl_toolkits.mplot3d import Axes3D
# from sklearn.cluster import DBSCAN

# data = np.array([[-37.530, 3.109, -16.452],
#                 [40.247, 5.483, -15.209],
#                 [-31.920, 12.584, -12.916],
#                 [-32.760, 14.072, -13.749],
#                 [-37.100, 1.953, -15.720],
#                 [-32.143, 12.990, -13.488],
#                 [-41.077, 4.651, -15.651], 
#                 [-34.219, 13.611, -13.090],
#                 [-33.117, 15.875, -13.738]])

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(data[:,0], data[:,1], data[:,2], s=300);
# ax.view_init(azim=200);
# plt.show();

# model = DBSCAN(eps=2.5, min_samples=2)
# model.fit_predict(data)
# pred = model.fit_predict(data)

# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(data[:,0], data[:,1], data[:,2], c=model.labels_, s=3)
# #ax.view_init(azim=200)
# plt.show()

# print("number of cluster found: {}".format(len(set(model.labels_))))
# print('cluster for each point: ', model.labels_)

In [ ]:

# from sklearn.cluster import DBSCAN
# from mpl_toolkits.mplot3d import Axes3D
# from sklearn.preprocessing  import MinMaxScaler

# for i in UNIQUE_ROIS:
#     print (i)
#     roi_num = i
#     hull = hulls[roi_num]

#     points = hull.points

#     points = points[:, [2, 1, 0]]

#     datapoints = df[df["roi"] == roi_num]
#     db_df = datapoints[["z", "y", "x"]]
#     db_df_np=np.array(db_df)
#     db_df_np
    
#     X=db_df_np
#     scaler = MinMaxScaler()
#     scaler.fit(X)
#     X_norm = scaler.transform(X)
#     db_df_np=X_norm

#     model = DBSCAN(eps=0.05, min_samples = 30,leaf_size=30)
#     #model.fit_predict( db_df_np)
#     pred = model.fit_predict( db_df_np)
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.scatter(db_df_np[:,0], db_df_np[:,1], db_df_np[:,2], c=model.labels_, s=1)
#     #ax.view_init(azim=200)
#     plt.show()

#     print("number of cluster found: {}".format(len(set(model.labels_))))
#     #print('cluster for each point: ', model.labels_)

In [ ]:
# #using optics 
# #https://scikit-learn.org/dev/modules/generated/sklearn.cluster.OPTICS.html

# from sklearn.cluster import DBSCAN
# from mpl_toolkits.mplot3d import Axes3D
# from sklearn.preprocessing  import MinMaxScaler
# from sklearn.cluster import OPTICS

# for i in UNIQUE_ROIS:
#     print (i)
#     roi_num = i
#     hull = hulls[roi_num]

#     points = hull.points

#     points = points[:, [2, 1, 0]]

#     datapoints = df[df["roi"] == roi_num]
#     db_df = datapoints[["z", "y", "x"]]
#     db_df_np=np.array(db_df)
#     db_df_np
    
#     X=db_df_np
#     scaler = MinMaxScaler()
#     scaler.fit(X)
#     X_norm = scaler.transform(X)
#     db_df_np=X_norm

# #     model = DBSCAN(eps=0.05, min_samples = 30,leaf_size=30)
# #     #model.fit_predict( db_df_np)
# #     pred = model.fit_predict( db_df_np)
#     clustering = OPTICS(min_samples=8).fit(db_df_np)
    
#     fig = plt.figure()
#     ax = fig.add_subplot(111, projection='3d')
#     ax.scatter(db_df_np[:,0], db_df_np[:,1], db_df_np[:,2], c=clustering.labels_, s=1)
#     #ax.view_init(azim=200)
#     plt.show()

#     print("number of cluster found: {}".format(len(set(clustering.labels_))))
#     #print('cluster for each point: ', model.labels_)

In [ ]:
    from sklearn.cluster import DBSCAN
    from mpl_toolkits.mplot3d import Axes3D
    from sklearn.preprocessing  import MinMaxScaler
    from sklearn.cluster import OPTICS

    roi_num = 3
    hull = hulls[roi_num]

    points = hull.points

    points = points[:, [2, 1, 0]]

    datapoints = df[df["roi"] == roi_num]
    db_df = datapoints[["z", "y", "x"]]
    db_df_np=np.array(db_df)
    db_df_np
    
#     X=db_df_np
#     scaler = MinMaxScaler()
#     scaler.fit(X)
#     X_norm = scaler.transform(X)
#     db_df_np=X_norm
    db_df_np=X

#     model = DBSCAN(eps=0.05, min_samples = 30,leaf_size=30)
#     #model.fit_predict( db_df_np)
#     pred = model.fit_predict( db_df_np)
    clustering = OPTICS(min_samples=15).fit(db_df_np)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.scatter(db_df_np[:,0], db_df_np[:,1], db_df_np[:,2], c=clustering.labels_, s=1)
    #ax.view_init(azim=200)
    plt.show()

    print("number of cluster found: {}".format(len(set(clustering.labels_))))
    #print('cluster for each point: ', model.labels_)

In [ ]:
df_export = pd.DataFrame(db_df_np)
df_export["clusters"] = clustering.labels_

In [ ]:
    datapoints=   df_export
    datapoints = datapoints.set_axis(['z', 'y', 'x', 'scalar'], axis=1)
    datapoints

In [ ]:

    d = {'x': datapoints['x'],'y': datapoints['y'],'z': datapoints['z'], "scalar":datapoints['scalar'] }
    cloud = PyntCloud(pd.DataFrame(data=d))
    cloud.to_file(out+str(roi_num)+"output_TEST.ply")
    datapoints.to_csv(out+str(roi_num)+"point_cloud_cluster.csv", index=False)

In [ ]:


import open3d as o3d
xyz = point_cloud
pcd = o3d.geometry.PointCloud()
#pcd.points = o3d.utility.Vector3dVector(xyz)
#pcd.colors = o3d.utility.Vector3dVector(scalar)

In [ ]:
# vis = o3d.visualization.Visualizer()
# vis.create_window()
# vis.add_geometry(pcd)
# o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.8)
# vis.run()

In [ ]:
#!pip install threejs

In [ ]:
# #3D_unet
# #Create movie from base_path (search folders)
# import os
# from glob import glob
# import moviepy.video.io.ImageSequenceClip

# output_base = "D:\\AMR_SIM\\AMR_POINT_CLOUD_WORKFLOW\\PYOUT\\"
# movie_out="D:\\AMR_SIM\\AMR_POINT_CLOUD_WORKFLOW\\movie_out\\"
# fps=3

# image_files = [y for x in os.walk(output_base) for y in glob(os.path.join(x[0], '*.png'))]
# image_files = [word for word in image_files if '3D_model' in word]
# clip = moviepy.video.io.ImageSequenceClip.ImageSequenceClip(image_files, fps=fps)
# clip.write_videofile(movie_out+'amr_jan_varicosities.mp4')

In [ ]:
#!pip install moviepy --upgrade

In [ ]:
    roi_num = 3
    hull = hulls[roi_num]

    points = hull.points

    points = points[:, [2, 1, 0]]

    datapoints = df[df["roi"] == roi_num]
    
    d = {'x': datapoints['x'],'y': datapoints['y'],'z': datapoints['z']}
    cloud = PyntCloud(pd.DataFrame(data=d))
    cloud.to_file(out+str(roi_num)+"output.ply")

    #hull to mesh
    vertices = points[hull.vertices]  # Vertices of the convex hull
    faces = hull.simplices  # Faces of the convex hull

    hull_mesh = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))

    for i, face in enumerate(faces):
        for j in range(3):
            hull_mesh.vectors[i][j] = points[face[j]]  #


    hull_mesh.save(out+str(roi_num)+"convex_hull_mesh.stl")

    hull_mesh_STL=pv.read(out+str(roi_num)+"convex_hull_mesh.stl")
    points_ply=pv.read(out+str(roi_num)+"output.ply")



    pv.set_plot_theme("document")
    p = pv.Plotter()
    p.set_background("white")
    p.add_mesh(hull_mesh_STL, color="grey", opacity=0.1)
    p.add_points(points_ply, style='points_gaussian',point_size=2, color="red")
    #p.add_mesh(plastic_STL, color="grey", opacity=0.1);
    p.window_size = 500, 500
    p.show(jupyter_backend='static');
    p.screenshot(out+str(roi_num)+'varicosities_3D_model.png') ;